<a href="https://colab.research.google.com/github/DajanaSanchez/BirdMigrationClimateAnalysis/blob/main/climate_data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

# Unmount Google Drive
drive.flush_and_unmount()

# Remount Google Drive
drive.mount('/content/drive', force_remount=True)

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [ ]:
import pandas as pd
import os
import tarfile
import glob
from google.colab import files

# Define file paths in Google Drive
BASE_DIR = "/content/drive/MyDrive/climate_data/"
PRECIP_DATA = os.path.join(BASE_DIR, "ghcn-m_v4.00.00_prcp_s16970101_e20241231_c20250105.tar")
PRECIP_INVENTORY = os.path.join(BASE_DIR, "ghcn-m_v4_prcp_inventory.txt")
TEMP_METADATA = os.path.join(BASE_DIR, "ghcnm.tavg.v4.0.1.20250127.qfe.inv")
TEMP_DATA = os.path.join(BASE_DIR, "ghcnm.tavg.v4.0.1.20250127.qfe.dat")

In [ ]:
#counting the number of files in the tar raw data file that start with MX, CA, US for validating purposes
import tarfile
import os

VALID_COUNTRIES = {'CA', 'US', 'MX'}

def count_country_files_in_tar(tar_path):
    """ Count files by country code in the tar archive """
    country_counts = {code: 0 for code in VALID_COUNTRIES}

    with tarfile.open(tar_path, "r:") as tar:
        for member in tar.getmembers():
            if member.name.endswith('.csv'):
                country_code = os.path.basename(member.name)[:2]
                if country_code in country_counts:
                    country_counts[country_code] += 1

    print("📊 File count by country:")
    for country, count in country_counts.items():
        print(f"{country}: {count} files")

    return country_counts


count_country_files_in_tar(PRECIP_DATA)

📊 File count by country:
CA: 8785 files
US: 63213 files
MX: 5314 files


{'CA': 8785, 'US': 63213, 'MX': 5314}

In [ ]:
# Deletes all extracted files
import shutil
shutil.rmtree("/content/precip_data", ignore_errors=True)

In [ ]:
#extracting US, CA, MX CSV files from raw data tar file
import os
import tarfile
import pandas as pd

VALID_COUNTRIES = {'CA', 'US', 'MX'}

#Extracts only CSV files for selected countries while ensuring a consistent 11-column structure.

def extract_tar_fixed(tar_path, extract_to="/content/drive/MyDrive/climate_data/precip_data"):

    if not os.path.exists(extract_to):
        os.makedirs(extract_to)

    print(f"📂 Extracting files from {tar_path}...")

    extracted_files = []  # Track extracted files

    with tarfile.open(tar_path, "r:*") as tar:
        for member in tar.getmembers():
            filename = os.path.basename(member.name)

            # ✅ Check if the file belongs to a valid country and is a CSV
            if filename.endswith('.csv') and filename[:2] in VALID_COUNTRIES:
                tar.extract(member, path=extract_to)
                extracted_file_path = os.path.join(extract_to, filename)

                # 🚀 Read file in a way that avoids parser errors
                try:
                    df = pd.read_csv(extracted_file_path, header=None, dtype=str, on_bad_lines='skip')

                    # 🚨 Identify files with inconsistent columns
                    if df.shape[1] == 12:
                        print(f"⚠️ {filename} has 12 columns! Fixing extra Station Name column...")
                        df.drop(columns=[2], inplace=True)  # Drop second column

                    # Ensure it has exactly 11 columns
                    if df.shape[1] != 11:
                      print(f"❌ {filename} still has {df.shape[1]} columns! Check manually.")

                    # ✅ Save the cleaned file
                    df.to_csv(extracted_file_path, index=False, header=False)
                    extracted_files.append(filename)

                except pd.errors.ParserError as e:
                    print(f"❌ Parser error in {filename}: {e}")
                    continue  # Skip this file

    print(f"✅ Successfully extracted and standardized {len(extracted_files)} files with 11 columns.")
    return extract_to

extracted_precip = extract_tar_fixed(PRECIP_DATA, extract_to="/content/precip_data")

📂 Extracting files from /content/drive/MyDrive/climate_data/ghcn-m_v4.00.00_prcp_s16970101_e20241231_c20250105.tar...
⚠️ CA004063753.csv has 12 columns! Fixing extra Station Name column...
⚠️ CA008103780.csv has 12 columns! Fixing extra Station Name column...
⚠️ CA008403045.csv has 12 columns! Fixing extra Station Name column...
⚠️ MXM00076258.csv has 12 columns! Fixing extra Station Name column...
⚠️ MXM00076382.csv has 12 columns! Fixing extra Station Name column...
⚠️ MXM00076525.csv has 12 columns! Fixing extra Station Name column...
⚠️ MXM00076685.csv has 12 columns! Fixing extra Station Name column...
❌ Parser error in MXM00076692.csv: Error tokenizing data. C error: EOF inside string starting at row 1169
⚠️ MXM00076741.csv has 12 columns! Fixing extra Station Name column...
⚠️ MXM00076775.csv has 12 columns! Fixing extra Station Name column...
⚠️ MXMLP076220.csv has 12 columns! Fixing extra Station Name column...
⚠️ MXMLP076373.csv has 12 columns! Fixing extra Station Name colum

In [ ]:
#validating structure of files that had 12 columns to ensure proper structure
import os

extract_to = "/content/precip_data"  # Change if the extracted files are in a different directory
file_name = "MXMLP076746.csv"  # Change this to the file you want to check

# Check if the file exists
file_path = os.path.join(extract_to, file_name)

if os.path.exists(file_path):
    print(f"📌 Found file: {file_path}")
else:
    print("❌ File not found. Check the filename or extraction path.")



if os.path.exists(file_path):
    df = pd.read_csv(file_path, header=None, dtype=str)

    print(f"📊 {file_name} has {df.shape[1]} columns and {df.shape[0]} rows.")
    print(df.head())  # Show the first few rows to verify structure
else:
    print("❌ File not found.")


📌 Found file: /content/precip_data/MXMLP076746.csv
📊 MXMLP076746.csv has 11 columns and 240 rows.
            0                                  1          2           3   \
0  MXMLP076746                        "CD OBREGON    18.5333    -92.6500   
1  MXMLP076746                        "CD OBREGON    18.5333    -92.6500   
2  MXMLP076746                        "CD OBREGON    18.5333    -92.6500   
3  MXMLP076746                        "CD OBREGON    18.5333    -92.6500   
4  MXMLP076746                        "CD OBREGON    18.5333    -92.6500   

         4       5       6  7  8  9       10  
0    -999.9  196101     342        R  111732  
1    -999.9  196102      -1        R  111732  
2    -999.9  196103      -1        R  111732  
3    -999.9  196104      -1        R  111732  
4    -999.9  196105      -1        R  111732  


In [ ]:
#moving extracted files to Google Drive
import shutil
import os

# Define paths
local_extracted_path = "/content/precip_data"
drive_destination_path = "/content/drive/MyDrive/climate_data/precip_data"

for filename in os.listdir(local_extracted_path):
    if filename.endswith(".csv"):
        src = os.path.join(local_extracted_path, filename)
        dst = os.path.join(drive_destination_path, filename)
        shutil.move(src, dst)

print(f"✅ Moved all extracted files to Google Drive: {drive_destination_path}")

In [ ]:
# Loading precipitation metadata and filtering to only include CA, MX, and US
import pandas as pd
import re

# List of valid country codes
VALID_COUNTRIES = {'CA', 'US', 'MX'}

def load_precip_metadata(file_path):
    print("Loading precipitation metadata...")
    try:
        data = []
        with open(file_path, 'r') as file:
            for line in file:
                match = re.match(r"^(\S+)\s+([\d\.-]+)\s+([\d\.-]+)\s+([\d\.-]+)\s+(\S{2})?\s+(.+?)\s+(\d{5})?\s+(\d{4})?\s+(\d{4})?$", line.strip())
                if match:
                    groups = list(match.groups())

                    # Extract country code from the first two characters of Station_ID
                    country_code = groups[0][:2]  # First two characters

                    # Keep only CA, MX, US stations
                    if country_code not in VALID_COUNTRIES:
                        continue  # Skip this record

                    if groups[4] is None or groups[4] not in VALID_COUNTRIES:
                        if groups[4] is not None:
                            groups[5] = groups[4] + " " + groups[5]
                        groups[4] = None

                    data.append(groups)
                else:
                    print(f"⚠ Warning: Skipping malformed line: {line.strip()}")

        col_names = ["Station_ID", "Latitude", "Longitude", "Elevation", "State", "Station_Name", "WMO_ID", "First_Year", "Last_Year"]
        df = pd.DataFrame(data, columns=col_names)
        df[["Latitude", "Longitude", "Elevation", "First_Year", "Last_Year"]] = df[["Latitude", "Longitude", "Elevation", "First_Year", "Last_Year"]].apply(pd.to_numeric, errors='coerce')

        print(f"✅ Successfully loaded {df.shape[0]} rows and {df.shape[1]} columns (filtered for CA, MX, US).")
        return df
    except Exception as e:
        print(f"❌ Error loading metadata: {e}")
        return None

# Run the function and load filtered precipitation metadata
precip_metadata = load_precip_metadata(PRECIP_INVENTORY)

# Define the output path in Google Drive
precip_metadata_file = "/content/drive/MyDrive/climate_data/precip_metadata_filtered.csv"

# Save the metadata DataFrame to a CSV file
precip_metadata.to_csv(precip_metadata_file, index=False)

print(f"✅ Filtered precipitation metadata saved to {precip_metadata_file}")


Loading precipitation metadata...
✅ Successfully loaded 77312 rows and 9 columns (filtered for CA, MX, US).
✅ Filtered precipitation metadata saved to /content/drive/MyDrive/climate_data/precip_metadata_filtered.csv


In [ ]:
#validating country codes in cleaned metadata file
print("Unique country codes in the filtered dataset:")
print(precip_metadata['Station_ID'].str[:2].unique())

Unique country codes in the filtered dataset:
['CA' 'MX' 'US']


In [ ]:
import os

processed_precip_file = "/content/drive/MyDrive/climate_data/processed_precipitation.csv"

# Delete the incorrect processed file
if os.path.exists(processed_precip_file):
    os.remove(processed_precip_file)
    print("✅ Deleted the old processed_precipitation.csv file.")
else:
    print("⚠️ No existing processed file found.")

⚠️ No existing processed file found.


In [ ]:
import os
import glob
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

# Define paths
local_extracted_path = "/content/precip_data"  # Directory where extracted files are stored
output_csv_path = "/content/merged_precip_data.csv"  # Final merged CSV file
error_log = "/content/error_log.txt"  # Log file for issues

# List all CSV files in the directory
csv_files = glob.glob(os.path.join(local_extracted_path, "*.csv"))

print(f"📂 Found {len(csv_files)} CSV files to merge.")


def process_file(file):
    """ Reads and processes a single CSV file in chunks, ensuring a valid 11-column structure. """
    try:
        chunk_list = []

        for chunk in pd.read_csv(file,
            header=None,
            dtype=str,
            engine="python",
            sep=",",
            quotechar='"',
            skipinitialspace=True,
            chunksize=500000  # Process large files in chunks
        ):

            # 🚨 Assign Column Names
            chunk.columns = [
                "Station_ID", "Station_Name", "Latitude", "Longitude", "Elevation",
                "YearMonth", "Precip_Value", "Measurement_Flag", "QC_Flag", "Source_Flag", "Source_Index"
            ]

            # 🚨 Convert Precip_Value to mm, replace -1 values
            chunk["Precip_Value"] = pd.to_numeric(chunk["Precip_Value"], errors="coerce").replace(-1, 0) / 10

            # 🚨 Format YearMonth as YYYY-MM
            chunk["YearMonth"] = chunk["YearMonth"].str.strip()  # Remove spaces
            chunk["YearMonth"] = chunk["YearMonth"].apply(lambda x: f"{x[:4]}-{x[4:]}" if len(x) == 6 else x)

            chunk_list.append(chunk)

        return pd.concat(chunk_list, ignore_index=True) if chunk_list else None

    except Exception as e:
        with open(error_log, "a") as log:
            log.write(f"{file} - Error: {e}\n")
        print(f"❌ Error processing file {file}: {e}")
        return None


def merge_precipitation_data_parallel(directory, output_filename="merged_precip_data.csv", num_workers=4):
    """ Merges multiple precipitation files in parallel, logs errors, and saves a single CSV file. """

    output_file = os.path.join("/content", output_filename)

    all_files = glob.glob(os.path.join(directory, "**", "*.csv"), recursive=True)
    if not all_files:
        print(f"❌ No CSV files found in {directory}")
        return None

    print(f"🚀 Starting parallel merging with {num_workers} threads...")

    first_write = not os.path.exists(output_file)  # Write header only if file doesn't exist

    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        future_to_file = {executor.submit(process_file, file): file for file in all_files}

        for future in as_completed(future_to_file):
            file = future_to_file[future]
            try:
                df = future.result()
                if df is not None:
                    df.to_csv(output_file, mode='a', index=False, header=first_write)
                    first_write = False  # Ensure header is written only once
                    print(f"✅ Processed and merged: {file}")

            except Exception as e:
                with open(error_log, "a") as log:
                    log.write(f"{file} - Processing failed: {e}\n")
                print(f"❌ Failed to merge {file}: {e}")

    # 🚀 Ensure the error log is properly saved
    if os.path.exists(error_log):
        print(f"⚠️ Error log saved at {error_log}")
    else:
        print("❌ Error log not found. Something went wrong.")

    print(f"✅ Merging complete! Data saved to {output_file}")
    return output_file


# Run the merging function on local files
merged_precip_file = merge_precipitation_data_parallel(local_extracted_path, num_workers=4)

Streaming output truncated to the last 5000 lines.
✅ Processed and merged: /content/precip_data/MXN00015200.csv
✅ Processed and merged: /content/precip_data/USC00515675.csv
✅ Processed and merged: /content/precip_data/US1HIHI0086.csv
✅ Processed and merged: /content/precip_data/USC00299820.csv
✅ Processed and merged: /content/precip_data/US1TXCML117.csv
✅ Processed and merged: /content/precip_data/USC00350897.csv
✅ Processed and merged: /content/precip_data/US1ORCC0108.csv
✅ Processed and merged: /content/precip_data/US1SDED0013.csv
✅ Processed and merged: /content/precip_data/US1WAKP0048.csv
✅ Processed and merged: /content/precip_data/USC00413472.csv
✅ Processed and merged: /content/precip_data/USC00041967.csv
✅ Processed and merged: /content/precip_data/US1COBA0029.csv
✅ Processed and merged: /content/precip_data/US1ILLG0015.csv
✅ Processed and merged: /content/precip_data/US1RIPR0053.csv
✅ Processed and merged: /content/precip_data/CA004024120.csv
✅ Processed and merged: /content/p

In [ ]:
#validating station IDs all start with MX, CA, and US after merge.
import pandas as pd

# Load the processed precipitation dataset
processed_precip_file = "/content/merged_precip_data.csv"
df = pd.read_csv(processed_precip_file, dtype=str)

# Ensure Station_ID column is cleaned
df["Station_ID"] = df["Station_ID"].str.strip()

# Define valid prefixes
VALID_PREFIXES = ("MX", "CA", "US")

# Find rows where Station_ID doesn't start with valid prefixes
invalid_ids = df[~df["Station_ID"].str.startswith(VALID_PREFIXES, na=False)]

# Display results
print(f"🚨 Found {invalid_ids.shape[0]} invalid Station_IDs.")
print(invalid_ids[["Station_ID", "Station_Name"]].head(20))  # Show sample

invalid_ids.to_csv("/content/drive/MyDrive/climate_data/invalid_station_ids.csv", index=False)
print("📁 Invalid Station_IDs saved to Google Drive.")

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


🚨 Found 0 invalid Station_IDs.
Empty DataFrame
Columns: [Station_ID, Station_Name]
Index: []
📁 Invalid Station_IDs saved to Google Drive.


In [ ]:
#integrating metadata with raw data
import pandas as pd

# Define file paths
precip_data_file = "/content/merged_precip_data.csv"
precip_metadata_file = "/content/drive/MyDrive/climate_data/precip_metadata_filtered.csv"

# Read large precipitation file in chunks
chunksize = 500_000  # Adjust based on available RAM

precip_chunks = []
for chunk in pd.read_csv(precip_data_file, chunksize=chunksize):
    precip_chunks.append(chunk)

# Merge all chunks into a final DataFrame
precip_df = pd.concat(precip_chunks, ignore_index=True)
print(f"✅ Loaded precipitation data: {precip_df.shape}")

# Load station metadata
metadata_df = pd.read_csv(precip_metadata_file)

✅ Loaded precipitation data: (16378268, 11)


In [ ]:
#viewing samples of df
precip_df.head()

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Station_ID,Station_Name,Latitude,Longitude,Elevation,YearMonth,Precip_Value,Measurement_Flag,QC_Flag,Source_Flag,Source_Index
0,USC00336853,"""OH PROCTORVILLE DAM 27""",38.45,-82.3167,171.0,1952-01,154.9,NaN,NaN,D,64867.0
1,USC00336853,"""OH PROCTORVILLE DAM 27""",38.45,-82.3167,171.0,1952-02,46.1,NaN,NaN,D,64867.0
2,USC00336853,"""OH PROCTORVILLE DAM 27""",38.45,-82.3167,171.0,1952-03,130.6,NaN,NaN,D,64867.0
3,USC00336853,"""OH PROCTORVILLE DAM 27""",38.45,-82.3167,171.0,1952-04,93.2,NaN,NaN,D,64867.0
4,USC00336853,"""OH PROCTORVILLE DAM 27""",38.45,-82.3167,171.0,1952-05,151.9,NaN,NaN,D,64867.0


In [ ]:
#viewing samples of df
metadata_df.head()

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Station_ID,Latitude,Longitude,Elevation,State,Station_Name,WMO_ID,First_Year,Last_Year
0,CA001010066,48.8667,-123.2833,4.0,NaN,BC ACTIVE PASS,99999,1984,1996
1,CA001010235,48.4000,-123.4833,17.0,NaN,BC ALBERT HEAD,99999,1971,1995
2,CA001010595,48.5833,-123.5167,85.0,NaN,BC BAMBERTON OCEAN CEMENT,99999,1961,1980
3,CA001010720,48.5000,-124.0000,351.0,NaN,BC BEAR CREEK,99999,1910,1971
4,CA001010774,48.5000,-123.3500,61.0,NaN,BC BEAVER LAKE,99999,1894,1952


In [ ]:
# Merge metadata with precipitation data
precip_final = precip_df.merge(metadata_df, on="Station_ID", how="left")

# Save the enriched dataset
precip_final_file = "/content/drive/MyDrive/climate_data/precip_final.csv"
precip_final.to_csv(precip_final_file, index=False)

print(f"✅ Merged dataset saved to {precip_final_file}")

✅ Merged dataset saved to /content/drive/MyDrive/climate_data/precip_final.csv


In [ ]:
#viewing random samples
precip_final.sample(5)

,Station_ID,Station_Name_x,Latitude_x,Longitude_x,Elevation_x,YearMonth,Precip_Value,Measurement_Flag,QC_Flag,Source_Flag,Source_Index,Latitude_y,Longitude_y,Elevation_y,State,Station_Name_y,WMO_ID,First_Year,Last_Year
9392505,USS0011G30S,"""ID SEDGWICK PEAK SNOTEL""",42.5200,-111.9600,2392.7,199408,48.1,,,D,73745,42.5200,-111.9600,2392.7,NaN,ID SEDGWICK PEAK SNOTEL,99999.0,1988.0,2024.0
1984348,USC00116558,"""IL PALESTINE HCN""",38.9994,-87.6139,136.6,190207,41.4,,,D,55277,38.9994,-87.6139,136.6,NaN,IL PALESTINE,99999.0,1882.0,2023.0
13099774,CA003023280,"""AB HILLSDOWN""",52.1667,-113.5167,896.0,190801,6.7,,,D,25975,52.1667,-113.5167,896.0,NaN,AB HILLSDOWN,99999.0,1904.0,1960.0
6463178,USC00358797,"""OR VALE HCN""",43.9814,-117.2439,682.8,194405,19.5,,,D,66081,43.9814,-117.2439,682.8,NaN,OR VALE,99999.0,1891.0,2013.0
11037550,US1GABN0001,"""GA BALDWIN 1.4 ESE""",34.4821,-83.5293,403.9,201810,117.4,,,D,999999,34.4821,-83.5293,403.9,NaN,GA BALDWIN 1.4 ESE,99999.0,2008.0,2024.0


In [ ]:
#validating merged datasets
import pandas as pd
# Find Station_IDs missing in metadata
precip_data = pd.read_csv("/content/merged_precip_data.csv", usecols=["Station_ID"], dtype=str)
precip_metadata = pd.read_csv("/content/drive/MyDrive/climate_data/precip_metadata_filtered.csv", usecols=["Station_ID"], dtype=str)

missing_stations = precip_data[~precip_data["Station_ID"].isin(precip_metadata["Station_ID"])]
print(f"🚨 Missing Station_IDs in Metadata: {missing_stations.shape[0]}")
if not missing_stations.empty:
    print(missing_stations.head())


🚨 Missing Station_IDs in Metadata: 0


In [ ]:
#loading temperature metadata
import pandas as pd

# Define column widths based on the expected format
colspecs = [(0, 11), (12, 20), (21, 30), (31, 37), (38, None)]  # Adjust as needed

# Define column names
metadata_columns = ["Station_ID", "Latitude", "Longitude", "StnElev", "Station_Name"]

# Read the metadata file with fixed column widths
df_metadata = pd.read_fwf(TEMP_METADATA, colspecs=colspecs, header=None, names=metadata_columns)

# Convert Latitude and Longitude to float (to preserve negatives)
df_metadata["Latitude"] = pd.to_numeric(df_metadata["Latitude"], errors="coerce")
df_metadata["Longitude"] = pd.to_numeric(df_metadata["Longitude"], errors="coerce")

# Filter for stations in US, CA, MX only
df_metadata = df_metadata[df_metadata["Station_ID"].str.startswith(("US", "CA", "MX"))]

# Display the first few rows to verify longitude
print(df_metadata.head(10))


       Station_ID  Latitude  Longitude  StnElev               Station_Name
2404  CA001011500   48.9333  -123.7500     75.0                  CHEMAINUS
2405  CA001011920   48.5333  -123.3667     37.0                CORDOVA_BAY
2406  CA001011922   48.5167  -123.3667     26.0          CORDOVA_BAY_SOUTH
2407  CA001012008   48.4167  -123.7000     52.0                   COWICHAN
2408  CA001012010   48.7167  -123.5500      1.0  COWICHAN_BAY_CHERRY_POINT
2409  CA001012040   48.8167  -124.1333    177.0     COWICHAN_LAKE_FORESTRY
2410  CA001012055   48.8333  -124.0500    171.0              LAKE_COWICHAN
2411  CA001012475   48.4167  -123.2333     15.0           DISCOVERY_ISLAND
2412  CA001012550   48.7833  -123.7167      9.0                     DUNCAN
2413  CA001012570   48.7667  -123.6833      6.0            DUNCAN_FORESTRY


In [ ]:
#extracting temperatures
import pandas as pd
import re

# Define file paths
TEMP_DATA = "/content/drive/MyDrive/climate_data/ghcnm.tavg.v4.0.1.20250127.qfe.dat"
TEMP_OUTPUT_FILE = "/content/processed_temps_only.csv"

# Function to extract temperatures
def extract_temperatures(line):
    """Extracts station ID, year, and 12 temperature values."""

    station_id = line[:11].strip()

    # 🚀 Filter: Keep only "MX", "CA", or "US" stations
    if not station_id.startswith(("MX", "CA", "US")):
        return None

    year = line[11:15].strip()
    element = line[15:19].strip()

    # Only process TAVG rows
    if element != "TAVG":
        return None

    # Regex: Extract numerical temperature values (ignore flags)
    pattern = r"(-?\d{1,4})\s*[A-Za-z\"'{}\[\]|&]*"
    matches = re.findall(pattern, line[19:].strip())

    # Ensure exactly 12 temperature values
    if len(matches) != 12:
        return None

    # Convert temperature values to Celsius (divide by 100)
    temperatures = [int(temp) / 100.0 for temp in matches]

    return [station_id, year] + temperatures  # Store station, year, and monthly temperatures

# Read and parse the file
parsed_rows = []
with open(TEMP_DATA, "r") as file:
    for line in file:
        parsed_line = extract_temperatures(line)
        if parsed_line:
            parsed_rows.append(parsed_line)

# Define column names
column_names = ["Station_ID", "Year"] + [f"{m:02d}_Temp" for m in range(1, 13)]

# Create DataFrame
df_temps = pd.DataFrame(parsed_rows, columns=column_names)

# Save to CSV
df_temps.to_csv(TEMP_OUTPUT_FILE, index=False)

print(f"✅ Step 1 complete: Temperatures extracted! Data saved to {TEMP_OUTPUT_FILE}")
print(df_temps.head(10))  # Preview data

✅ Step 1 complete: Temperatures extracted! Data saved to /content/processed_temps_only.csv
    Station_ID  Year  01_Temp  02_Temp  03_Temp  04_Temp  05_Temp  06_Temp  \
0  CA001011500  1961     5.01     6.14     7.10     8.97    12.89    17.05   
1  CA001011500  1962     3.53     5.23     5.10     9.47    11.47    15.10   
2  CA001011500  1963     1.55     7.46     6.41     9.12    12.81    15.75   
3  CA001011500  1964     4.56     5.24     6.24     8.34    11.46    15.17   
4  CA001011500  1965     2.73     4.72     5.95     9.09    11.44    15.82   
5  CA001011500  1966     3.28     5.11     6.47     9.18    12.37    14.36   
6  CA001011500  1967     4.60     5.51     5.41     7.69    12.75    17.90   
7  CA001011500  1968     3.30     6.15     7.46     8.25    13.20    15.29   
8  CA001011500  1969    -1.80     3.29     6.38     8.31    14.03    18.46   
9  CA001011500  1970     3.62     6.45     7.03     8.22    11.79    17.11   

   07_Temp  08_Temp  09_Temp  10_Temp  11_Temp  12

In [ ]:
#extracting flags into a single field
FLAG_OUTPUT_FILE = "/content/processed_flags_only.csv"

# Function to extract flags correctly
def extract_flags(line):
    """Extracts 12 sets of flags from a line while keeping them as a single field."""

    station_id = line[:11].strip()

    # 🚀 Filter: Keep only "MX", "CA", or "US" stations
    if not station_id.startswith(("MX", "CA", "US")):
        return None

    year = line[11:15].strip()
    element = line[15:19].strip()

    # Only process TAVG rows
    if element != "TAVG":
        return None

    # Regex: Extract flag values (ignoring temperatures)
    pattern = r"-?\d{1,4}\s*([A-Za-z\"'{}\[\]|&]*)"
    matches = re.findall(pattern, line[19:].strip())

    # Ensure exactly 12 flags are extracted
    if len(matches) != 12:
        return None

    return [station_id, year] + matches

# Read and parse flags from the file
parsed_flags = []
with open(TEMP_DATA, "r") as file:
    for line in file:
        parsed_line = extract_flags(line)
        if parsed_line:
            parsed_flags.append(parsed_line)

# Define column names for flags
column_names_flags = ["Station_ID", "Year"] + [f"{m:02d}_Flags" for m in range(1, 13)]

# Create DataFrame for flags
df_flags = pd.DataFrame(parsed_flags, columns=column_names_flags)

# Save to CSV
df_flags.to_csv(FLAG_OUTPUT_FILE, index=False)

print(f"✅ Step 2 complete: Flags extracted! Data saved to {FLAG_OUTPUT_FILE}")
print(df_flags.head(10))  # Preview data

✅ Step 2 complete: Flags extracted! Data saved to /content/processed_flags_only.csv
    Station_ID  Year 01_Flags 02_Flags 03_Flags 04_Flags 05_Flags 06_Flags  \
0  CA001011500  1961        E        E        E        E        E        E   
1  CA001011500  1962        E        E        E        E        E        E   
2  CA001011500  1963        E        E        E        E        E        E   
3  CA001011500  1964        E        E        E        E        E        E   
4  CA001011500  1965        E        E        E        E        E        E   
5  CA001011500  1966        E        E        E        E        E        E   
6  CA001011500  1967        E        E        E        E        E        E   
7  CA001011500  1968        E        E        E        E        E        E   
8  CA001011500  1969        E        E        E        E        E        E   
9  CA001011500  1970        E        E        E        E        E        E   

  07_Flags 08_Flags 09_Flags 10_Flags 11_Flags 12_Flags  

In [ ]:
#sampling to ensure data looks accurate
df_flags.sample(10)

,Station_ID,Year,01_Flags,02_Flags,03_Flags,04_Flags,05_Flags,06_Flags,07_Flags,08_Flags,09_Flags,10_Flags,11_Flags,12_Flags
287572,USC00228530,1974,E,E,E,E,E,E,E,E,E,E,E,E
390210,USC00458903,2010,E,E,E,E,E,E,E,E,E,E,E,E
76867,CA004024160,1965,C,C,C,C,C,E,C,C,C,C,C,C
115998,CA006131415,1985,C,C,C,C,C,C,C,C,C,C,C,C
362131,USC00402957,1964,E,E,E,E,E,E,E,E,E,E,E,E
160784,CA008103050,1975,C,C,C,C,C,C,C,C,C,C,C,C
18208,CA001105155,1980,E,E,E,E,E,E,E,E,E,E,E,E
140393,CA007034395,1976,E,E,E,E,E,C,C,E,C,C,C,C
336683,USC00341900,1987,E,E,E,E,E,E,E,E,E,E,E,E
514567,USW00012826,1969,EXX,EXX,EXX,EXX,EXX,EXX,EXX,EXX,EXX,EXX,EXX,EXX


In [ ]:
#merging temperature and flags
MERGED_OUTPUT_FILE = "/content/processed_temperature_data.csv"

# Load extracted temperature & flag datasets
df_temps = pd.read_csv(TEMP_OUTPUT_FILE)
df_flags = pd.read_csv(FLAG_OUTPUT_FILE)

# Merge temperature and flag data on Station_ID and Year
df_final = df_temps.merge(df_flags, on=["Station_ID", "Year"])

# 🚀 Apply filtering again just in case
df_final = df_final[df_final["Station_ID"].str.startswith(("MX", "CA", "US"))]

# Save merged dataset
df_final.to_csv(MERGED_OUTPUT_FILE, index=False)

print(f"✅ Step 3 complete: Data merged and saved to {MERGED_OUTPUT_FILE}")
print(df_final.head(10))  # Preview final structured output

✅ Step 3 complete: Data merged and saved to /content/processed_temperature_data.csv
    Station_ID  Year  01_Temp  02_Temp  03_Temp  04_Temp  05_Temp  06_Temp  \
0  CA001011500  1961     5.01     6.14     7.10     8.97    12.89    17.05   
1  CA001011500  1962     3.53     5.23     5.10     9.47    11.47    15.10   
2  CA001011500  1963     1.55     7.46     6.41     9.12    12.81    15.75   
3  CA001011500  1964     4.56     5.24     6.24     8.34    11.46    15.17   
4  CA001011500  1965     2.73     4.72     5.95     9.09    11.44    15.82   
5  CA001011500  1966     3.28     5.11     6.47     9.18    12.37    14.36   
6  CA001011500  1967     4.60     5.51     5.41     7.69    12.75    17.90   
7  CA001011500  1968     3.30     6.15     7.46     8.25    13.20    15.29   
8  CA001011500  1969    -1.80     3.29     6.38     8.31    14.03    18.46   
9  CA001011500  1970     3.62     6.45     7.03     8.22    11.79    17.11   

   07_Temp  08_Temp  ...  03_Flags  04_Flags  05_Flags  0

In [ ]:
#pivoting data into long format

# 🚀 Load Processed Data
df = pd.read_csv(MERGED_OUTPUT_FILE)

# 🚀 Apply Final Filter (Just to be Safe)
df = df[df["Station_ID"].str.startswith(("MX", "CA", "US"))]

# 🚀 Pivot Temperature Data
df_temp_long = df.melt(
    id_vars=["Station_ID", "Year"],
    value_vars=[f"{m:02d}_Temp" for m in range(1, 13)],
    var_name="Month",
    value_name="Temperature"
)

# 🚀 Pivot Flags Data
df_flags_long = df.melt(
    id_vars=["Station_ID", "Year"],
    value_vars=[f"{m:02d}_Flags" for m in range(1, 13)],
    var_name="Month",
    value_name="Flags"
)

# 🚀 Extract Month Number
df_temp_long["Month"] = df_temp_long["Month"].str[:2]
df_flags_long["Month"] = df_flags_long["Month"].str[:2]


# 🚀 Merge Temperatures & Flags
df_final = df_temp_long.merge(df_flags_long, on=["Station_ID", "Year", "Month"], how="left")

# 🚀 Create Final Date Column
df_final["Date"] = df_final["Year"].astype(str) + "-" + df_final["Month"]

# 🚀 Drop Unnecessary Columns
df_final = df_final.drop(columns=["Year", "Month"])

# 🚀 Save Final Structured Data
FINAL_OUTPUT_FILE = "/content/final_climate_data.csv"
df_final.to_csv(FINAL_OUTPUT_FILE, index=False)

print("✅ Transformation complete! Data saved to final_climate_data.csv")
print(df_final.head(10))  # Preview final structured output


✅ Transformation complete! Data saved to final_climate_data.csv
    Station_ID  Temperature Flags     Date
0  CA001011500         5.01     E  1961-01
1  CA001011500         3.53     E  1962-01
2  CA001011500         1.55     E  1963-01
3  CA001011500         4.56     E  1964-01
4  CA001011500         2.73     E  1965-01
5  CA001011500         3.28     E  1966-01
6  CA001011500         4.60     E  1967-01
7  CA001011500         3.30     E  1968-01
8  CA001011500        -1.80     E  1969-01
9  CA001011500         3.62     E  1970-01


In [ ]:
df_final.sample(10)

,Station_ID,Temperature,Flags,Date
3687974,USC00408540,24.07,E,1965-07
3763241,USR0000CWWC,13.97,E,1971-07
6456682,USC00406740,1.95,E,1968-12
2709449,USS0013C39S,7.77,T,2007-05
4651975,USC00047714,22.10,E,1979-09
4540064,CA006104165,17.61,C,1968-09
5544639,CA001035614,7.72,E,1962-11
1972928,USC00293840,5.69,E,2002-04
1699086,CA002101135,-0.98,E,1961-04
2793803,CA001128983,14.58,E,1981-06


In [ ]:
#saving merged dataset as CSV
df_final.to_csv("/content/drive/MyDrive/climate_data/temp_final.csv", index=False)
print("✅ Merged dataset saved as temp_final.csv")

df_final.head()

✅ Merged dataset saved as temp_final.csv


,Station_ID,Temperature,Flags,Date
0,CA001011500,5.01,E,1961-01
1,CA001011500,3.53,E,1962-01
2,CA001011500,1.55,E,1963-01
3,CA001011500,4.56,E,1964-01
4,CA001011500,2.73,E,1965-01


In [ ]:
import pandas as pd

# Define file paths
temp_final_file = "/content/drive/MyDrive/climate_data/temp_final.csv"
precip_final_file = "/content/drive/MyDrive/climate_data/precip_final.csv"
merged_output_file = "/content/drive/MyDrive/climate_data/climate_merged_final.csv"

# 🚀 Load Datasets
df_temp = pd.read_csv(temp_final_file)
df_precip = pd.read_csv(precip_final_file)

# 🚀 Ensure Date Fields Match
df_temp.rename(columns={"Date": "YearMonth"}, inplace=True)  # Match date column name
df_precip["YearMonth"] = df_precip["YearMonth"].astype(str)  # Ensure it's a string

# Ensure Latitude and Longitude have four decimal place
df_precip["Latitude_y"] = df_precip["Latitude_y"].astype(float).map(lambda x: f"{x:.4f}")
df_precip["Longitude_y"] = df_precip["Longitude_y"].astype(float).map(lambda x: f"{x:.4f}")


# 🚀 Merge on Station_ID & Date (YearMonth)
df_merged = df_temp.merge(df_precip, on=["Station_ID", "YearMonth"], how="inner")

# 🚀 Rename Columns for Clarity
df_merged.rename(columns={
    "YearMonth": "Date",
    "Latitude_y": "Latitude",
    "Longitude_y": "Longitude",
    "Elevation_y": "Elevation",
    "Station_Name_y": "StationName",
    "Temperature": "Temp_celsius",
    "Precip_Value": "Precip_mm",
    "Flags": "TempFlags",
    "Measurement_Flag": "Precip_MeasurementFlag",
    "QC_Flag": "Precip_QCFlag",
    "Source_Flag": "Precip_SourceFlag",
    "Source_Index": "Precip_SourceIndex",
    "First_Year": "Precip_FirstYear",
    "Last_Year": "Precip_LastYear"
}, inplace=True)

# 🚀 Select Only Required Columns
df_merged = df_merged[[
    "Station_ID", "Date", "Latitude", "Longitude", "Elevation", "State", "StationName",
    "Temp_celsius", "Precip_mm", "TempFlags", "Precip_MeasurementFlag", "Precip_QCFlag",
    "Precip_SourceFlag", "Precip_SourceIndex", "Precip_FirstYear", "Precip_LastYear"
]]

# 🚀 Save Final Merged Dataset
df_merged.to_csv(merged_output_file, index=False)

print(f"✅ Merging complete! Data saved to {merged_output_file}")
print(df_merged.head(10))  # Preview final structured output

<ipython-input-16-6880b8112527>:10: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df_precip = pd.read_csv(precip_final_file)


✅ Merging complete! Data saved to /content/drive/MyDrive/climate_data/climate_merged_final.csv
    Station_ID     Date Latitude  Longitude  Elevation State   StationName  \
0  CA001011500  1961-01  48.9333  -123.7500       75.0   NaN  BC CHEMAINUS   
1  CA001011500  1962-01  48.9333  -123.7500       75.0   NaN  BC CHEMAINUS   
2  CA001011500  1963-01  48.9333  -123.7500       75.0   NaN  BC CHEMAINUS   
3  CA001011500  1964-01  48.9333  -123.7500       75.0   NaN  BC CHEMAINUS   
4  CA001011500  1965-01  48.9333  -123.7500       75.0   NaN  BC CHEMAINUS   
5  CA001011500  1966-01  48.9333  -123.7500       75.0   NaN  BC CHEMAINUS   
6  CA001011500  1967-01  48.9333  -123.7500       75.0   NaN  BC CHEMAINUS   
7  CA001011500  1968-01  48.9333  -123.7500       75.0   NaN  BC CHEMAINUS   
8  CA001011500  1969-01  48.9333  -123.7500       75.0   NaN  BC CHEMAINUS   
9  CA001011500  1970-01  48.9333  -123.7500       75.0   NaN  BC CHEMAINUS   

   Temp_celsius  Precip_mm TempFlags Precip_Me

In [ ]:
df_merged.sample(10)

,Station_ID,Date,Latitude,Longitude,Elevation,State,StationName,Temp_celsius,Precip_mm,TempFlags,Precip_MeasurementFlag,Precip_QCFlag,Precip_SourceFlag,Precip_SourceIndex,Precip_FirstYear,Precip_LastYear
1128486,CA001195250,1986-10,58.9333,-125.7667,837.0,NaN,BC MUNCHO LAKE,3.62,90.5,C,NaN,NaN,D,25164.0,1970,2013
1041576,CA00701S001,1997-09,46.8000,-71.3833,60.0,NaN,QC QUEBEC/JEAN LESAGE INTL,11.89,72.3,C,A,NaN,D,29742.0,1993,2024
586564,USS0005N16S,1997-05,36.9900,-105.2600,3230.9,NaN,NM NORTH COSTILLA SNOTEL,4.94,91.3,T,NaN,NaN,D,73467.0,1979,2024
777402,CA00503B1ER,2005-07,50.1833,-96.0667,268.0,NaN,MB PINAWA,18.92,86.0,C,NaN,NaN,D,27987.0,1992,2024
67071,CA008205990,1965-01,45.3667,-63.2667,40.0,NaN,NS TRURO,-7.53,86.4,C,NaN,NaN,D,31029.0,1960,2002
1308635,CA008401550,1980-11,48.7667,-56.6000,154.0,NaN,NL EXPLOITS DAM,1.52,135.2,C,NaN,NaN,D,31188.0,1956,2006
786839,CA006141095,1985-07,43.3333,-80.3167,268.0,NaN,ON CAMBRIDGE GALT MOE,19.23,70.2,C,NaN,NaN,D,29128.0,1879,1993
946876,USC00175736,2004-08,44.6353,-70.0003,146.3,NaN,ME NEW SHARON,18.93,151.0,E,NaN,NaN,D,57877.0,1993,2024
544436,CA007026836,1961-05,45.1333,-74.3500,53.0,NaN,QC ST ANICET,11.27,60.5,C,NaN,NaN,D,29886.0,1960,2024
401541,CA005021220,1997-04,49.0333,-97.5667,253.0,NaN,MB GRETNA (AUT),-0.78,11.6,C,D,NaN,D,27794.0,1885,2024
